In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

**Importing The Libraries**

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

from sklearn import metrics

**Reading the Dataset - 'Blue Books for Bulldozers' for auction prize prediction.**

In [3]:
data_raw = pd.read_csv('../input/Train.csv',low_memory=False,parse_dates=['saledate'])

**Viewing the First Few Entries**

In [4]:
data_raw.head().transpose()

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
SalePrice,66000,57000,10000,38500,11000
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneerID,3,3,3,3,3
YearMade,2004,1996,2001,2001,2007
MachineHoursCurrentMeter,68,4640,2838,3486,722
UsageBand,Low,Low,High,High,Medium
saledate,2006-11-16 00:00:00,2004-03-26 00:00:00,2004-02-26 00:00:00,2011-05-19 00:00:00,2009-07-23 00:00:00


**Problem - Predicting the sale price of bulldozers sold at auctions.**

*Contest Details - 
The goal of the contest is to predict the sale price of a particular piece of heavy equiment at auction based on it's usage, equipment type, and configuaration.  The data is sourced from auction result postings and includes information on usage and equipment configurations.*

*Fast Iron is creating a "blue book for bull dozers," for customers to value what their heavy equipment fleet is worth at auction. *

**Evaluation Criteria:**
*The evaluation metric for this competition is the RMSLE (root mean squared log error) between the actual and predicted auction prices.*

In [5]:
data_raw.SalePrice = np.log(data_raw.SalePrice)

In [6]:
data_raw.SalePrice.head()

0    11.097410
1    10.950807
2     9.210340
3    10.558414
4     9.305651
Name: SalePrice, dtype: float64

In [7]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(data_raw.drop(columns=['SalePrice'],axis=1),data_raw.SalePrice)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ValueError: could not convert string to float: 'Conventional'

It appears that we have categorical data in our dataset

In [8]:
data_raw.saledate.head()

0   2006-11-16
1   2004-03-26
2   2004-02-26
3   2011-05-19
4   2009-07-23
Name: saledate, dtype: datetime64[ns]

*Extracting useful information from **saledate** column like date, month, year, day_of_week, is_month_start, is_month_end, is_year_start, is_year_end etc.*

In [9]:
add_datepart(data_raw,'saledate')

In [10]:
data_raw.saleDayofweek.head()

0    3
1    4
2    3
3    3
4    3
Name: saleDayofweek, dtype: int64

In [11]:
data_raw.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'saleYear', 'saleMonth',
       'saleWeek', 'saleDay', 'saleDayofweek', 'saleDayofyear',


From above we can find out that sale date is converted into more number of useful columns.

In [12]:
data_raw.shape[1]

65

*Using the : **train_cats(dataframe)** to Change any columns of strings in a panda's dataframe to a column of
categorical values. This applies the changes inplace.*

In [13]:
train_cats(data_raw)

Lets see into how many categories UsageBand is classified into

In [15]:
data_raw.UsageBand.cat.categories

Index(['High', 'Low', 'Medium'], dtype='object')

Arranging the above categories into more sensible way.

In [16]:
data_raw.UsageBand.cat.set_categories(['High','Medium','Low'],ordered=True,inplace=True)

In [17]:
data_raw.UsageBand.cat.codes.head()

0    2
1    2
2    0
3    0
4    1
dtype: int8

**Checking For Missing Values**

In [18]:
data_raw.isnull().sum().sort_index()/len(data_raw.SalePrice)

Backhoe_Mounting            0.803872
Blade_Extension             0.937129
Blade_Type                  0.800977
Blade_Width                 0.937129
Coupler                     0.466620
Coupler_System              0.891660
Differential_Type           0.826959
Drive_System                0.739829
Enclosure                   0.000810
Enclosure_Type              0.937129
Engine_Horsepower           0.937129
Forks                       0.521154
Grouser_Tracks              0.891899
Grouser_Type                0.752813
Hydraulics                  0.200823
Hydraulics_Flow             0.891899
MachineHoursCurrentMeter    0.644089
MachineID                   0.000000
ModelID                     0.000000
Pad_Type                    0.802720
Pattern_Changer             0.752651
ProductGroup                0.000000
ProductGroupDesc            0.000000
ProductSize                 0.525460
Pushblock                   0.937129
Ride_Control                0.629527
Ripper                      0.740388
S

In [20]:
df_raw = data_raw

*Now we have separated SalePrice column from df_raw and returned dataframe into df, and SalePrice(Target Variable) into y. Along with handling missing values and categorical data*

In [21]:
df, y,nac = proc_df(df_raw,'SalePrice')

In [24]:
df.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,saleYear,saleMonth,saleWeek,saleDay,saleDayofweek,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed,auctioneerID_na,MachineHoursCurrentMeter_na
0,1139246,999089,3157,121,3.0,2004,68.0,3,950,296,41,0,0,0,59,1,6,6,0,3,1,0,2,0,0,0,0,0,0,0,1,0,0,0,0,17,3,0,0,0,0,0,0,0,0,0,0,0,0,4,2,2006,11,46,16,3,320,False,False,False,False,False,False,1163635200,False,False
1,1139248,117657,77,121,3.0,1996,4640.0,3,1725,527,55,98,0,4,62,33,6,6,0,3,1,0,2,0,0,0,0,0,0,0,1,0,0,0,0,12,3,0,0,0,0,0,0,0,0,0,0,0,0,4,2,2004,3,13,26,4,86,False,False,False,False,False,False,1080259200,False,False
2,1139249,434808,7009,121,3.0,2001,2838.0,1,331,110,0,0,0,0,39,32,3,3,0,6,1,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,3,1,1,3,0,0,0,0,0,0,0,0,0,0,0,2004,2,9,26,3,57,False,False,False,False,False,False,1077753600,False,False
3,1139251,1026470,332,121,3.0,2001,3486.0,1,3674,1375,0,45,0,6,8,44,4,4,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2011,5,20,19,3,139,False,False,False,False,False,False,1305763200,False,False
4,1139253,1057373,17311,121,3.0,2007,722.0,2,4208,1529,0,0,0,0,40,32,3,3,0,1,1,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,3,1,1,3,0,0,0,0,0,0,0,0,0,0,0,2009,7,30,23,3,204,False,False,False,False,False,False,1248307200,False,False


**Lets Build our First Model using Random Forest with 10 estimators**

In [25]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df,y)
m.score(df,y)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9831226840988966

*Lets split the df and y into train sets and validation sets to check whether model overfits on validation set*

In [26]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()
n_valid = 12000  # same as Kaggle's test set size
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)
X_train.shape, y_train.shape, X_valid.shape

((389125, 66), (389125,), (12000, 66))

**Lets define the Root Mean Squared Error Function**

In [27]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

**print_score(model)** *Function to print r^2 score and rmse on train set and validation set*

In [28]:
def print_score(m):
    res = [rmse(m.predict(X_train), y_train),
           rmse(m.predict(X_valid), y_valid),
           m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [29]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


CPU times: user 1min 39s, sys: 764 ms, total: 1min 39s
Wall time: 29.7 s
[0.09051145093430334, 0.2480872696338315, 0.9828784972546952, 0.8900850079458388]


**Lets Tune the Hyper parameters of our Model**

In [31]:
set_rf_samples(50000)

**set_rf_samples(n)**:
*Changes Scikit learn's random forests to give each tree a random sample of n*

In [32]:
m = RandomForestRegressor(n_estimators = 100,n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

CPU times: user 3min 15s, sys: 1.52 s, total: 3min 17s
Wall time: 51.6 s
[0.1944310252429609, 0.2470933687390086, 0.9209929295647407, 0.8909639388013467]


**Our Model is not too much overfitting on the training set.**
**RMSE on training set:     0.1944**
**RMSE on validation set: 0.2470
**

In [36]:
set_rf_samples(50000)

In [37]:
m = RandomForestRegressor(n_estimators = 100,n_jobs=-1,min_samples_leaf=3)
%time m.fit(X_train, y_train)
print_score(m)

CPU times: user 2min 48s, sys: 464 ms, total: 2min 49s
Wall time: 43.8 s
[0.2035506565391983, 0.24647218549240466, 0.9134075871678974, 0.8915114746401221]


**We obtained a better performance on Validation set and our model is now not overfitting much**

* RMSE on training set: 0.2035
* R^2 Score on training set: 0.9134
* RMSE on validation set: 0.2464
* R^2 Score on validation set: 0.8915